In [1]:
import math
import numpy as np
import sys
import theano
sys.path.append('../code/')
import AEPDGP_net
import time

sys.path.append('/homes/mlghomes/mh740/Doubly-Stochastic-DGP/demos')
from get_data import get_regression_data

print 'Theano version: ' + theano.__version__ + ', base compile dir: ' + theano.config.base_compiledir
# theano.config.compute_test_value = 'warn'
theano.config.optimizer = 'None'
# theano.exception_verbosity = 'high'
theano.config.mode = 'FAST_RUN'
theano.config.optimizer = 'fast_run'
np.random.seed(1234)

Couldn't import dot_parser, loading of dot files will not be possible.
Theano version: 0.9.0, base compile dir: /home/mh740/.theano


In [2]:
X, Y, Xs, Ys = get_regression_data('kin8nm', split=0)
Y = Y.reshape((-1))
Ys = Ys.reshape((-1))
print 'N: {}, D: {}, Ns: {}'.format(X.shape[0], X.shape[1], Xs.shape[0])

N: 7372, D: 8, Ns: 820


In [3]:
# number of GP layers
nolayers = 2
# number of hidden dimension in intermediate hidden layers
n_hiddens = [3]
# number of inducing points per layer
M = 50
n_pseudos = [M for _ in range(nolayers)]

no_iterations = 1000
no_points_per_mb = 500

# We construct the network
net = AEPDGP_net.AEPDGP_net(X, Y, n_hiddens, n_pseudos)
t0 = time.time()
# train
test_nll, test_rms, energy = net.train(no_iterations=no_iterations,
                               no_points_per_mb=no_points_per_mb,
                               lrate=0.01)

<<!! BUG IN FGRAPH.REPLACE OR A LISTENER !!>> <type 'exceptions.TypeError'> ('The type of the replacement must be compatible with the type of the original Variable.', Sum{acc_dtype=int64}.0, Elemwise{mul,no_inplace}.0, TensorType(int64, scalar), TensorType(bool, scalar), 'local_opt_alloc') local_opt_alloc
ERROR (theano.gof.opt): Optimization failure due to: local_opt_alloc
ERROR (theano.gof.opt): node: Sum{acc_dtype=int64}(Alloc.0)
ERROR (theano.gof.opt): TRACEBACK:
ERROR (theano.gof.opt): Traceback (most recent call last):
  File "/usr/local/lib/python2.7/dist-packages/theano/gof/opt.py", line 2022, in process_node
    remove=remove)
  File "/usr/local/lib/python2.7/dist-packages/theano/gof/toolbox.py", line 391, in replace_all_validate_remove
    chk = fgraph.replace_all_validate(replacements, reason)
  File "/usr/local/lib/python2.7/dist-packages/theano/gof/toolbox.py", line 340, in replace_all_validate
    fgraph.replace(r, new_r, reason=reason, verbose=False)
  File "/usr/local/li

training iter: 10, logZ: 5388.08756
training iter: 20, logZ: 3811.29999
training iter: 30, logZ: 3838.51347
training iter: 40, logZ: 3140.22555
training iter: 50, logZ: 2473.04126
training iter: 60, logZ: 1555.78839
training iter: 70, logZ: 967.08499
training iter: 80, logZ: 138.48234
training iter: 90, logZ: -554.28426
training iter: 100, logZ: -628.21918
training iter: 110, logZ: -1043.19466
training iter: 120, logZ: -1701.31865
training iter: 130, logZ: -1730.27144
training iter: 140, logZ: -1896.38718
training iter: 150, logZ: -1874.57777
training iter: 160, logZ: -2401.57603
training iter: 170, logZ: -2291.94931
training iter: 180, logZ: -2461.41255
training iter: 190, logZ: -2714.63945
training iter: 200, logZ: -2662.11462
training iter: 210, logZ: -3091.51547
training iter: 220, logZ: -2992.74381
training iter: 230, logZ: -2881.72595
training iter: 240, logZ: -2899.56053
training iter: 250, logZ: -3312.43425
training iter: 260, logZ: -2985.08958
training iter: 270, logZ: -3231.3

In [4]:
t1 = time.time()
print 'time: ', t1 - t0

# We make predictions for the test set
m, v = net.predict(Xs)

# We compute the test RMSE
rmse = np.sqrt(np.mean((Ys - m)**2))
print 'test rmse: ', rmse

# We compute the test log-likelihood
test_ll = np.mean(-0.5 * np.log(2 * math.pi * (v)) - \
    0.5 * (Ys - m)**2 / (v))
print 'test log-likelihood: ', test_ll

time:  2071.75235605
test rmse:  0.0853715558013
test log-likelihood:  1.05178101939
